In [1]:
import numpy as np  # sometimes needed to avoid mkl-service error
import sys
import os
import argparse
import logging
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.utilities import rank_zero_only
import torch
from torchmdnet.module import LNNP
from torchmdnet import datasets, priors, models
from torchmdnet.data import DataModule
from torchmdnet.models import output_modules
from torchmdnet.models.utils import rbf_class_mapping, act_class_mapping
from torchmdnet.utils import LoadFromFile, LoadFromCheckpoint, save_argparse, number
from pathlib import Path
import wandb
import json
import pandas as pd
from rdkit.Chem import AllChem
import copy
from rdkit.Geometry import Point3D

In [2]:
from rdkit import Chem
suppl = Chem.SDMolSupplier("data\\qm9\\raw\\gdb9.sdf", removeHs=False,
                                   sanitize=True)

In [5]:
len(suppl)

133885

In [179]:
type(suppl[18981])
suppl[105]==None

False

In [85]:
smi = Chem.MolToSmiles(suppl[18982])
#smi = "C1C[C@H]2CC[C@@H]12"
smi

'[H]N1C2([H])C3([H])OC([H])([H])OC2([H])C13[H]'

In [81]:
params = Chem.SmilesParserParams()
params.removeHs = False
target_mol=Chem.MolFromSmiles(smi,params)
#target_mol.UpdatePropertyCache(strict=False)
#target_mol=Chem.AddHs(target_mol)
Chem.MolToSmiles(target_mol)

'[H]C1([H])C([H])([H])[C@]2([H])C([H])([H])C([H])([H])[C@]12[H]'

In [80]:
smi="[H]C1([H])C([H])([H])[C@]2([H])C([H])([H])C([H])([H])[C@]12[H]"
asdasd="[H]C1([H])C([H])([H])[C@]2([H])C([H])([H])C([H])([H])[C@]12[H]"
asdasd

'[H]C1([H])C([H])([H])[C@]2([H])C([H])([H])C([H])([H])[C@]12[H]'

In [88]:
mol=suppl[18982]

In [3]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import rdkit
print(rdkit.__version__)

2022.03.1


In [86]:
ps = rdDistGeom.ETKDGv3()
#ps.randomSeed = 0xf01d
ps.trackFailures = True
target_mol=Chem.MolFromSmiles(smi,params)
target_mol=Chem.AddHs(target_mol)
a=-1
while a!=0:
    a=rdDistGeom.EmbedMolecule(target_mol,ps)
    print(a)
conf=target_mol.GetConformer()
target_pos=conf.GetPositions()

0


In [131]:
a=rdDistGeom.EmbedMolecule(target_mol,ps)

In [75]:

noise=np.random.randn(target_pos.shape[0],target_pos.shape[1]) 
newpositions=target_pos+noise
for i in range(mol.GetNumAtoms()):
    conf.SetAtomPosition(i, Point3D(newpositions[i][0],newpositions[i][1],newpositions[i][2]))

In [89]:
Chem.rdMolAlign.AlignMol(mol,target_mol)

RuntimeError: No sub-structure match found between the probe and query mol

In [163]:
a=target_mol.GetConformers()
a[0]

IndexError: End of sequence hit

In [91]:
mol=suppl[375]
positions=mol.GetConformer().GetPositions()


In [91]:
Chem.rdMolAlign.GetAlignmentTransform(mol,target_mol)

RuntimeError: No sub-structure match found between the probe and query mol

In [95]:
Chem.rdMolAlign.CalcRMS(mol,target_mol,0,0)

RuntimeError: No sub-structure match found between the reference and probe mol

In [96]:
Chem.rdMolAlign.GetBestRMS(mol,target_mol,0,0)

RuntimeError: No sub-structure match found between the reference and probe mol

In [109]:
Chem.rdForceFieldHelpers.UFFOptimizeMolecule(target_mol,confId=0,maxIters=40)

1

In [4]:
IPythonConsole.drawMol3D(suppl[0],confId=0)
#IPythonConsole.drawMol3D(target_mol,confId=0)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom
import rdkit



In [12]:
cf=mol2.GetConformer()
IPythonConsole.drawMol3D(mol2,confId=0)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
IPythonConsole.drawMol3D(mol,confId=0)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
#Chem.AddHs(mol)
from rdkit.Geometry import Point3D
conf=mol.GetConformer(0)
print(conf.GetId())
print(conf.GetPositions())
positions=conf.GetPositions()
noise=np.random.randn(positions.shape[0],positions.shape[1]) * 0.03
newpositions=positions+noise
for i in range(mol.GetNumAtoms()):
    conf.SetAtomPosition(i, Point3D(newpositions[i][0],newpositions[i][1],newpositions[i][2]))
print(conf.GetPositions())
Chem.rdForceFieldHelpers.MMFFOptimizeMolecule(mol,confId=0,maxIters=1000)
print(conf.GetPositions())


0
[[ 0.0448  1.4745  0.082 ]
 [-0.1141  0.0382 -0.1087]
 [ 0.676  -0.5269 -1.1998]
 [ 2.209  -0.5299 -1.0082]
 [ 0.2077 -1.9009 -1.4573]
 [-0.1284 -2.9894 -1.6489]
 [ 1.0681  1.8091  0.3181]
 [-0.2775  1.9965 -0.8259]
 [-0.6101  1.7952  0.8967]
 [ 0.0798 -0.4635  0.7515]
 [ 0.4397  0.0608 -2.0973]
 [ 2.5728  0.491  -0.8688]
 [ 2.4841 -1.1238 -0.1315]
 [ 2.7102 -0.9567 -1.8814]]
[[ 0.04386523  1.52345625  0.09775143]
 [-0.14531104  0.01013604 -0.09918746]
 [ 0.75045109 -0.5286323  -1.18670165]
 [ 2.18794694 -0.5979397  -0.98743622]
 [ 0.22980846 -1.89373686 -1.54160074]
 [-0.09860824 -2.98607448 -1.57854657]
 [ 1.05860199  1.81894906  0.32410607]
 [-0.24731624  1.98949928 -0.82270865]
 [-0.58530139  1.80543866  0.86149709]
 [ 0.10120597 -0.4983125   0.73739685]
 [ 0.43927168  0.05895377 -2.09809234]
 [ 2.56048581  0.51042521 -0.90809486]
 [ 2.49636246 -1.16184422 -0.06867952]
 [ 2.7664228  -0.94920639 -1.8966374 ]]
[[ 0.0663793   1.46643826  0.06130053]
 [-0.08941926  0.02706983 -0.0749

In [7]:
Chem.rdMolAlign.GetBestRMS(mol,mol2,0,0)

0.062077115809664865

In [6]:
Chem.rdMolAlign.CalcRMS(mol,mol2,0,0)

0.06615768525565952

In [8]:
mol.GetConformer(0).GetPositions()

array([[ 0.05872255,  1.4677921 ,  0.07522191],
       [-0.1028098 ,  0.03041105, -0.07463314],
       [ 0.63975791, -0.52745663, -1.2206198 ],
       [ 2.1544128 , -0.50720707, -0.99133927],
       [ 0.21277532, -1.92546269, -1.49393268],
       [-0.09565279, -3.02482825, -1.70888307],
       [ 1.08183834,  1.74161525,  0.34817623],
       [-0.22437412,  1.99887786, -0.8391899 ],
       [-0.59977981,  1.82242352,  0.87448684],
       [ 0.17846828, -0.44417442,  0.7835278 ],
       [ 0.40965343,  0.06188993, -2.11734768],
       [ 2.53698934,  0.51427335, -0.90119888],
       [ 2.43091233, -1.05651489, -0.08353927],
       [ 2.68118623, -0.97743911, -1.83022909]])

In [9]:
mol2.GetConformer(0).GetPositions()

array([[ 0.0448,  1.4745,  0.082 ],
       [-0.1141,  0.0382, -0.1087],
       [ 0.676 , -0.5269, -1.1998],
       [ 2.209 , -0.5299, -1.0082],
       [ 0.2077, -1.9009, -1.4573],
       [-0.1284, -2.9894, -1.6489],
       [ 1.0681,  1.8091,  0.3181],
       [-0.2775,  1.9965, -0.8259],
       [-0.6101,  1.7952,  0.8967],
       [ 0.0798, -0.4635,  0.7515],
       [ 0.4397,  0.0608, -2.0973],
       [ 2.5728,  0.491 , -0.8688],
       [ 2.4841, -1.1238, -0.1315],
       [ 2.7102, -0.9567, -1.8814]])

In [6]:
conf.SetId(2)

In [ ]:

IPythonConsole.drawMol3D(mol)


In [4]:
from rdkit.Chem import AllChem
import numpy as np
mol = Chem.MolFromSmiles(smi)
mol=Chem.AddHs(mol)

In [13]:
Chem.AllChem.EmbedMolecule(mol)
conf=mol.GetConformer()
conf.GetPositions()

array([[ 0.27115779, -0.0235985 , -1.03320013],
       [ 1.58941196, -0.04171642, -0.58475614],
       [-0.77393345,  0.23685482, -0.14215826],
       [-0.74429438,  1.56223022,  0.48034737],
       [-0.70170269,  2.61720748,  0.98265005],
       [-0.95651482, -0.82290911,  0.94254558],
       [ 0.0559249 , -0.22666395, -2.00024404],
       [ 2.17859399, -0.75726254, -1.23374429],
       [ 2.15208176,  0.93111531, -0.69847639],
       [ 1.75734462, -0.39537658,  0.44630028],
       [-1.74340917,  0.20915221, -0.72064797],
       [-0.19416651, -0.73072232,  1.7410267 ],
       [-0.90098491, -1.82591115,  0.47921811],
       [-1.98950909, -0.73239947,  1.34113914]])

In [15]:
IPythonConsole.drawMol3D(mol,confId=0)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [109]:
from rdkit.Chem import AllChem
import numpy as np
mol = Chem.MolFromSmiles(smi)
mol=Chem.AddHs(mol)
mol=Chem.AllChem.EmbedMolecule(mol)

conf=mol.GetConformer()
positions=conf.GetPositions()
noise=np.random.randn(positions.shape[0],positions.shape[1]) * 0.01
newpositions=positions+noise
atoms = mol.GetAtoms()
print(positions)
print(newpositions)
from rdkit.Geometry import Point3D
cmap={}
for i in range(mol.GetNumAtoms()):
     x,y,z = newpositions[i]
     cmap[i]=Point3D(x,y,z)
# # Print the 3D coordinates.
# for i, atom in enumerate(mol.GetAtoms()):
#         positions = mol.GetConformer().GetAtomPosition(i)
#         print(atom.GetSymbol(), positions.x, positions.y, positions.z)
print(cmap)
print(Chem.AllChem.EmbedMolecule(mol,coordMap=cmap))
conf=mol.GetConformer(1)
newpositions=conf.GetPositions()
print(positions)
print(newpositions)
Chem.AllChem.GetConformerRMS(mol,mol.GetConformer().GetId(),mol.GetConformer().GetId())

[[-0.57369486 -0.34726401 -0.63030103]
 [-1.63815228 -0.06481543  0.24955673]
 [ 0.74040383 -0.31213397 -0.10262216]
 [ 0.88112454 -1.36862999  0.89939833]
 [ 0.97445998 -2.22546326  1.71191055]
 [ 1.08596875  1.01043591  0.54176568]
 [-0.70120597 -0.22903322 -1.62643794]
 [-2.58245961 -0.4891844  -0.194668  ]
 [-1.47958462 -0.48668021  1.23712752]
 [-1.78990203  1.03582197  0.29881887]
 [ 1.44977685 -0.49642419 -0.96707976]
 [ 2.16746343  1.18180594  0.39564123]
 [ 0.53376092  1.85183858  0.13520076]
 [ 0.93204108  0.93972629  1.65727957]]
[[-0.56576933 -0.33996841 -0.6370018 ]
 [-1.64962335 -0.06590017  0.24434526]
 [ 0.73563061 -0.30406548 -0.07998088]
 [ 0.88747604 -1.36506486  0.91551934]
 [ 0.9742831  -2.23762431  1.69908215]
 [ 1.09369423  0.9890999   0.51240133]
 [-0.69733596 -0.22119061 -1.61783139]
 [-2.56544638 -0.47826326 -0.19671292]
 [-1.46465324 -0.49553262  1.24104491]
 [-1.79032885  1.04474068  0.30212459]
 [ 1.44408854 -0.49170089 -0.94544525]
 [ 2.14653272  1.1934589

In [20]:
N = suppl[0].GetNumAtoms()
pos = suppl.GetItemText(0).split('\n')[4:4 + N]
pos = [[float(x) for x in line.split()[:3]] for line in pos]
pos = torch.tensor(pos, dtype=torch.float)

In [25]:
molblock = Chem.MolBlock(pos.tolist())
mol = Chem.MolFromMolBlock(molblock)

AttributeError: module 'rdkit.Chem' has no attribute 'MolBlock'

In [ ]:
rdkit.Chem.MolFromCoords(coordinates)

In [3]:

with open('commandline_args.txt', 'r') as f:
    args = json.load(f)

In [4]:
data = DataModule(args)
data.prepare_data()
data.setup("fit")
print(data.dataset_maybe_noisy[0].pos_target)
data.setup_target_pos()
print(data.dataset_maybe_noisy[0].pos_target)

c:\Users\aminr\.conda\envs\pvd\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train 400, val 50, test 41


computing mean and std: 100%|██████████| 4/4 [00:00<00:00, 11.67it/s]


tensor([[-0.6018,  0.0647,  0.2421],
        [-0.0609,  0.1834, -0.2531],
        [ 0.3118, -0.1557, -0.3511],
        [ 0.1198,  0.6480, -0.0855],
        [ 0.6533,  0.1511, -0.0563]])
tensor([[-0.2248,  0.1975, -0.1273],
        [-0.0005, -0.1091, -0.0276],
        [-0.1158,  0.1861, -0.0848],
        [ 0.2729,  0.2275, -0.1764],
        [ 0.1117,  0.2643, -0.2115]])


In [13]:
data.dataset[0].


AttributeError: 'GlobalStorage' object has no attribute 'target'

In [13]:
prior = None
if args["prior_model"]:
    assert hasattr(priors, args.prior_model), (
        f"Unknown prior model {args['prior_model']}. "
        f"Available models are {', '.join(priors.__all__)}"
    )
    # initialize the prior model
    prior = getattr(priors, args.prior_model)(dataset=data.dataset)
    args.prior_args = prior.get_init_args()

In [6]:
mol_name_id={}
for id in data.dataset.indices():
    mol_name_id[data.dataset[id].name]=id
filename="data/qm9/raw/gdb9.sdf.ref.csv"
df=pd.read_csv(filename)
df=df.set_index("mol_id")

In [7]:
for id in data.dataset.indices():
    name=data.dataset[id].name
    ref_name=df.loc[name]["ref_mol_id"]
    ref_mol_id=mol_name_id[ref_name]
    data.dataset[id].pos_target=data.dataset[ref_mol_id].pos

In [9]:
data.dataset[0].pos_target

AttributeError: 'GlobalStorage' object has no attribute 'pos_target'

In [106]:
mol_name_id.keys()

dict_keys(['gdb_237', 'gdb_307', 'gdb_261', 'gdb_142', 'gdb_382', 'gdb_294', 'gdb_298', 'gdb_30', 'gdb_156', 'gdb_154', 'gdb_443', 'gdb_145', 'gdb_36', 'gdb_310', 'gdb_111', 'gdb_326', 'gdb_233', 'gdb_496', 'gdb_138', 'gdb_407', 'gdb_40', 'gdb_478', 'gdb_216', 'gdb_378', 'gdb_197', 'gdb_450', 'gdb_143', 'gdb_74', 'gdb_16', 'gdb_37', 'gdb_479', 'gdb_248', 'gdb_350', 'gdb_127', 'gdb_500', 'gdb_119', 'gdb_239', 'gdb_139', 'gdb_419', 'gdb_125', 'gdb_191', 'gdb_463', 'gdb_11', 'gdb_256', 'gdb_149', 'gdb_17', 'gdb_413', 'gdb_10', 'gdb_195', 'gdb_229', 'gdb_291', 'gdb_246', 'gdb_336', 'gdb_144', 'gdb_70', 'gdb_438', 'gdb_242', 'gdb_206', 'gdb_498', 'gdb_271', 'gdb_456', 'gdb_7', 'gdb_140', 'gdb_488', 'gdb_259', 'gdb_180', 'gdb_2', 'gdb_357', 'gdb_453', 'gdb_440', 'gdb_386', 'gdb_424', 'gdb_278', 'gdb_340', 'gdb_6', 'gdb_290', 'gdb_109', 'gdb_171', 'gdb_313', 'gdb_94', 'gdb_157', 'gdb_442', 'gdb_103', 'gdb_408', 'gdb_270', 'gdb_179', 'gdb_235', 'gdb_155', 'gdb_175', 'gdb_46', 'gdb_400', 'gdb_2

In [115]:
for name in mol_name_id.keys():
    ref_name=df.loc[name]["ref_mol_id"]
    mol_id=mol_name_id[name]
    ref_mol_id=mol_name_id[ref_name]
    data.dataset[mol_id].pos_target=data.dataset[ref_mol_id].pos

In [116]:
data

In [14]:
model = LNNP(args, prior_model=prior, mean=data.mean, std=data.std)

In [3]:
data_list= torch.load("data/qm9/processed/data_v3.pt")

In [14]:
torch.save(data_list, "data/qm9/processed/data_v3.pt")

In [13]:
data_list[0]

Data(x=[2147303, 11], edge_index=[2, 4409754], edge_attr=[4409754, 4], pos=[2147303, 3], z=[2147303], name=[119136], idx=[119136], mol=[119136], noisy_mol=[119136])

In [141]:
a=torch.unbind(data_list[0])

TypeError: unbind(): argument 'input' (position 1) must be Tensor, not list

In [19]:
print(model)

LNNP(
  (model): TorchMD_Net(
    (representation_model): TorchMD_ET(hidden_channels=256, num_layers=8, num_rbf=64, rbf_type=expnorm, trainable_rbf=False, activation=silu, attn_activation=silu, neighbor_embedding=NeighborEmbeddingJittable_dd7ef0(
      (embedding): Embedding(100, 256)
      (distance_proj): Linear(in_features=64, out_features=256, bias=True)
      (combine): Linear(in_features=512, out_features=256, bias=True)
      (cutoff): CosineCutoff()
    ), num_heads=8, distance_influence=both, cutoff_lower=0.0, cutoff_upper=5.0)
    (output_model): EquivariantScalar(
      (output_network): ModuleList(
        (0): GatedEquivariantBlock(
          (vec1_proj): Linear(in_features=256, out_features=256, bias=False)
          (vec2_proj): Linear(in_features=256, out_features=128, bias=False)
          (update_net): Sequential(
            (0): Linear(in_features=512, out_features=256, bias=True)
            (1): SiLU()
            (2): Linear(in_features=256, out_features=256, bia